In [11]:
import numpy as np
import pandas as pd
import pymc3 as pm

In [58]:
alpha, sigma = 1, 1
beta = [1, 2.5]
N = 1000
X1 = np.random.randn(N)
X2 = np.random.randn(N) * 0.2
Y = alpha + beta[0] * X1 + beta[1] * X2 + np.random.randn(N) * sigma

with pm.Model() as model:
    alpha = pm.Normal('alpha', mu = 0, sd = 10)
    beta = pm.Normal('beta', mu = 0, sd = 10, shape = 2)
    sigma = pm.HalfNormal('sigma', sd = 1)
    mu = alpha + beta[0] * X1 + beta[1] * X2
    Y_obs = pm.Normal('Y_obs', mu = mu, sd = sigma, observed = Y)
    start = pm.find_MAP()
    trace = pm.sample(2000, start = start)
    
trace['alpha'][:5]

Applied log-transform to sigma and added transformed sigma_log_ to model.
Assigned NUTS to alpha
Assigned NUTS to beta
Assigned NUTS to sigma_log_
100%|██████████| 2000/2000 [00:04<00:00, 489.17it/s]


array([ 0.99959347,  0.99959347,  0.99143305,  0.99804502,  1.00746962])

In [63]:
# from
# https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers
# /blob/master/Chapter1_Introduction/Ch1_Introduction_PyMC3.ipynb

url = 'https://raw.githubusercontent.com/CamDavidsonPilon/'
url += 'Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/master/'
url += 'Chapter1_Introduction/data/txtdata.csv'
data = pd.read_table(url, header = None).iloc[:, 0].values
N = len(data)
alpha = 1.0 / data.mean()

with pm.Model() as model:
    lambda_1 = pm.Exponential('lambda_1', alpha)
    lambda_2 = pm.Exponential('lambda_2', alpha)
    tau = pm.DiscreteUniform('tau', lower = 0, upper = N)
    idx = np.arange(N)
    lambda_ = pm.math.switch(tau >= idx, lambda_1, lambda_2)
    observation = pm.Poisson('obs', lambda_, observed = data)
    step = pm.Metropolis()
    trace = pm.sample(10000, tune = 5000, step = step)    

Applied log-transform to lambda_1 and added transformed lambda_1_log_ to model.
Applied log-transform to lambda_2 and added transformed lambda_2_log_ to model.
100%|██████████| 10000/10000 [00:03<00:00, 2951.94it/s]
